# Flight Delay Prediction (Jan 2019)
The dataset contains records gathered by the Bureau of Transportation Statistics (BTS)[9] toprovide historical comparisons of monthly on-time reports filed by large US Airlines.  

_Only datasetsfor 2019 intentionally selected due to the substantial effect of COVID19 in the aviation industry in2020, it is out of scope of this project to analyze this factor._
### Source:Bureau of Transportation Statistics 
### Number of rows:1,984,933
### Dataset features and description:

| Variable name               | Description                                                                                            |
|-----------------------------|--------------------------------------------------------------------------------------------------------|
| Quarter (Time Period)       | Quarter (1-4)                                                                                          |
| Month                       | Month                                                                                                  |
| DayOfWeek                   | Day of Week  (1 - Monday, 2 -Tuesday, 3 - Wendsday)                                                                                          |
| Operating\_Airline          | Carrier Code                                                                                           |
| Origin                      | Origin Airport                                                                                         |
| Dest                        | Destination Airport                                                                                    |
| DepTime                     | Actual Departure Time (local time: hhmm)                                                               |
| DepDelay                    | Difference between scheduled and actual departure time (Minutes)                                       |
| DepDel15                    | Departure Delay Indicator (1=Yes)                                                                      |
| DepartureDelayGroups        | Departure Delay intervals, every (15 minutes from <-15 to >180)                                        |
| TaxiOut                     | Taxi-Out Time (Minutes)                                                                                |
| TaxiIn                      | Taxi-In Time (Minutes)                                                                                 |
| **ArrDelay (target)**          | **Difference in minutes between scheduled and actual arrival time**. *Early arrivals show negative numbers |
| ArrDel15                    | Arrival Delay Indicator, 15 Minutes or More (1=Yes)                                                    |
| ArrivalDelayGroups          | Arrival Delay intervals (15-minutes from <-15 to >180)                                                 |
| Cancelled                   | Cancelled Flight Indicator (1=Yes)                                                                     |
| CancellationCode            | Specifies The Reason For Cancellation                                                                  |
| ActualElapsedTime           | Elapsed Time of Flight, in Minutes                                                                     |
| AirTime                     | Flight Time, in Minutes                                                                                |
| Flights                     | Number of Flights                                                                                      |
| Distance                    | Distance between airports (Miles)                                                                      |
| DistanceGroup               | Distance FLight Segment, (every 250 Miles)                                                             |
| CarrierDelay                | Delay by Carrier (Minutes)                                                                             |
| NASDelay                    | Delay by NAS (Minutes)                                                                                 |
| SecurityDelay               | Delay by Security (Minutes)                                                                            |
| LateAircraftDelay           | Delay by Late Aircraft (in Minutes)                                                                    |
| WeatherDelay                | Delay caused by Weather (Minutes)                                                                      |


In [ ]:
#Import basic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [ ]:
#Setup pandas display parameters
pd.options.display.max_columns = 50
pd.options.display.max_rows = 200
pd.options.display.max_colwidth= 50
pd.options.display.precision = 3

In [ ]:
#Define initial model parameters
cv_n_split = 3
random_state = 42
test_train_split = 0.25
sample = True
sample_size = 0.4

In [ ]:
#data = pd.read_csv("../1019415451_T_ONTIME_MARKETING-4.csv")
df01_csv = pd.read_csv("../../Downloads/ontime-2019-01.csv")
print ("Imported {} with {} variables".format(df01_csv.shape[0],df01_csv.shape[1]))

In [ ]:
df01_csv.dtypes

In [ ]:
if sample:
    df01_csv = df01_csv.sample(frac = sample_size, replace=True, random_state=random_state)


# Data Cleaning 
* Selecting relevant and discard irrelevant columns for the prediction
* clean missing values
* Variable transformations
* Configuring up variable types

In [ ]:
#List available columns on dataset
list(df01_csv.columns) 

In [ ]:

#Select most relevant columns, drop the rest
relevant_columns = ['DAY_OF_MONTH','DAY_OF_WEEK', # Date information
                    'OP_CARRIER','TAIL_NUM', #Airline and Aircraft Identifier
                    'ORIGIN','DEST',#Origin and destination
                    'DEP_DELAY','DEP_DELAY_NEW','DEP_DEL15','DEP_DELAY_GROUP','DEP_TIME_BLK','TAXI_OUT', #departure delays
                    'ARR_DELAY','ARR_DELAY_NEW','ARR_DEL15','ARR_DELAY_GROUP','ARR_TIME_BLK',#Arrival information
                    'CRS_ELAPSED_TIME','ACTUAL_ELAPSED_TIME','AIR_TIME','FLIGHTS','DISTANCE','DISTANCE_GROUP',#Flight summaries
                    'CANCELLED','DIVERTED', #Cancelled/Deleted information
                    'CARRIER_DELAY','WEATHER_DELAY','NAS_DELAY','SECURITY_DELAY','LATE_AIRCRAFT_DELAY']
df01 = df01_csv[df01_csv.columns.intersection(relevant_columns)] #Extract relevant columns from csv dataframe
df01.columns = map(str.lower, df01.columns)  #Set column names to lowercase
df01.head()

In [ ]:
df01.dtypes

In [ ]:
#Review dataframe columns summary for diagnostics and cleaning
pd.DataFrame({ 
            'unicos':df01.nunique(),
            'missing total': df01.isna().sum(),
            'missing %': df01.isna().sum()/df01.count(),
            'type':df01.dtypes})

In [ ]:
df01.op_carrier.unique()

In [ ]:
df01.describe()

In [ ]:
outliers = df01[df01['arr_delay_new']>1000]
outliers.head()

In [ ]:
#Fix data types for bool columns
df01["diverted"] = df01["diverted"].apply(lambda x: 1 if x > 0 else 0)
df01["cancelled"] = df01["cancelled"].apply(lambda x: 1 if x > 0 else 0)
df01["arr_del15"] = df01["arr_del15"].apply(lambda x: 1 if x > 0 else 0)
df01["dep_del15"] = df01["dep_del15"].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
df01.head()

In [ ]:
#Fix data types for categorical columns
for col in ['day_of_month','day_of_week','op_carrier','origin','dest','dep_delay_group','dep_time_blk','arr_time_blk' ,'arr_delay_group','distance_group']:
    df01[col] = df01[col].astype('category')
#Fix data types for string columns
df01['tail_num'] = df01['tail_num'].astype('string')

In [ ]:
df01.dtypes

In [ ]:
df01.flights.value_counts()

In [ ]:
df_delayed = df01[(df01['cancelled'] == 0) & (df01['diverted']==0)]
df_delayed.shape

In [ ]:
#Replace with 0 all NA values on delay 
df_delayed['carrier_delay'] = df_delayed['carrier_delay'].fillna(0)
df_delayed['weather_delay'] = df_delayed['weather_delay'].fillna(0)
df_delayed['nas_delay'] = df_delayed['nas_delay'].fillna(0)
df_delayed['security_delay'] = df_delayed['security_delay'].fillna(0)
df_delayed['late_aircraft_delay'] = df_delayed['late_aircraft_delay'].fillna(0)

In [ ]:
df_delayed.drop(['flights','diverted','cancelled'], axis='columns', inplace=True)

In [ ]:
#Review dataframe columns summary for diagnostics and cleaning
pd.DataFrame({ 
            'unicos':df_delayed.nunique(),
            'missing total': df_delayed.isna().sum(),
            'missing %': df_delayed.isna().sum()/df_delayed.count(),
            'type':df_delayed.dtypes})

# Data transformations

In [ ]:
#Create variable to identify if flight is in a weekend
df_delayed['is_weekend'] = np.where(df_delayed['day_of_week'].isin([5,6,7]), 1, 0)

In [ ]:
df_delayed[df_delayed['day_of_week'].isin([5,6,7])]

In [ ]:
df_delayed['is_weekend'].value_counts()

In [ ]:
#Import dataset for bussi airports, contains airport code and Total Enplaned Passengers in 2019
top_airports_csv = pd.read_csv("../../Downloads/topAirports2.csv")
top_airports_csv.head()

In [ ]:
top_airports_csv['total_enplaned'].describe()

In [ ]:
plt.style.use('ggplot')
plt.hist(top_airports_csv['total_enplaned'],bins=20)
plt.show()

In [ ]:
top_airports_csv.plot(kind='bar',x='airport_code',y='total_enplaned')

In [ ]:
# Selecting the top 5 most busy airports by passenger traffic count
top5 = top_airports_csv.nlargest(10, 'total_enplaned')
top5['airport_code']

In [ ]:
df_delayed['origin_top5'] = df_delayed.origin.isin(top5['airport_code'].tolist())
df_delayed['dest_top5'] = df_delayed.dest.isin(top5['airport_code'].tolist())

In [ ]:
df_delayed.head()

In [ ]:
import seaborn as sns
sns.set()
sns.set_context('notebook',rc = {"grid.linewidth": 5})
sns.set_style("whitegrid")
colors = ["#345E6F","#264653","#287271","#2a9d8f","#e9c46a","#efb366","#f4a261","#ee8959","#e76f51","#e87153","#e97c61", '#902C14']
bin_colors = ["#264653","#2A9D8F","#ee8959","#e97c61"]
sns.set_palette(sns.color_palette(colors))
mul_palette = sns.color_palette(colors)
bin_palette = sns.color_palette(bin_colors)
sns.set(rc={'figure.figsize':(10,5)}, font_scale=1.5)
sns.set_style({'axes.facecolor':'white', 'grid.color': '.8','grid.linestyle': '--'})

In [ ]:
ax = sns.scatterplot(x="dep_delay_new", y="taxi_out", size='arr_delay_new', data = df_delayed)
ax.set(xlabel = "Departure Delay", ylabel = "Arrival Delay",title = 'Departure delay vs arrival delay (Minutes)')
#set labels friendly name
#ax.set_xticklabels(['On Time','Delayed'])
#Hide Y labels
#ax.axes.yaxis.set_visible(False)

In [ ]:
ax = sns.scatterplot(x="arr_time_blk", y="arr_delay_new", data = df_delayed)
ax.set(xlabel = "Departure Delay", ylabel = "Arrival Delay",title = 'Departure delay vs arrival delay (Minutes)')
#set labels friendly name
ax.set_xticklabels(list(df_delayed.arr_time_blk.unique()),rotation=90)
#Hide Y labels
#ax.axes.yaxis.set_visible(False)

In [ ]:
ax = sns.countplot(x="arr_time_blk", hue='arr_del15', data = df_delayed,palette = bin_palette)
ax.set(xlabel = "Departure Delay", ylabel = "Arrival Delay",title = 'Departure delay vs arrival delay (Minutes)')
#set labels friendly name
ax.set_xticklabels(list(df_delayed.arr_time_blk.unique()),rotation=90)

In [ ]:
#create plot
ax = sns.countplot(x = 'arr_del15', hue = 'day_of_week' , data = df_delayed,palette = mul_palette)
#Set Title
ax.set(xlabel = "",title = 'Delayed Flights per day of the week')
#set labels friendly name
ax.legend(['Monday','Tuesday','Wensday','Thursday','Friday','Saturday','Sunday'])
ax.set_xticklabels(['On Time','Delayed'])
#Hide Y labels
ax.axes.yaxis.set_visible(False)

In [ ]:
df_delayed[df_delayed['arr_del15']>0]

In [ ]:
#create plot
ax = sns.countplot(x = 'day_of_month' , data = df_delayed[df_delayed['arr_del15']>0],palette = mul_palette)
#Set Title
ax.set(xlabel = "",title = 'Delayed flights per day of the month')
#set labels friendly name
#ax.set_xticklabels(['On Time','Delayed'])
#Hide Y labels
ax.axes.yaxis.set_visible(False)

In [ ]:
sns.catplot(x="day_of_week", y="arr_delay", data=df_delayed, palette = mul_palette, kind="swarm")
ax.set(xlabel = "",title = 'Delayed time per day of week')
#set labels friendly name
#ax.set_xticklabels(['On Time','Delayed'])
ax.set_xticklabels(['Monday','Tuesday','Wensday','Thursday','Friday','Saturday','Sunday'],rotation=30)
#Hide Y labels
ax.axes.yaxis.set_visible(False)

## Column manipulations

In [ ]:
aircrafts = df_delayed.tail_num.unique()
print(aircrafts)

In [ ]:
len(aircrafts)

In [ ]:
df_aircrafts = df01[['tail_num']]
df_aircrafts = df_aircrafts.drop_duplicates(subset = ["tail_num"])
df_aircrafts.to_csv('df_aircrafts.csv',index=False)

In [ ]:
df_aircrafts.head(100)

In [ ]:
df_airports = df01.drop_duplicates(subset = ["origin"])
df_airports = df_airports[['origin']]
df_airports.head()

In [ ]:
df01.groupby(['dep_del15','origin']).size().unstack().plot(kind='bar',stacked=True)
plt.show()

In [ ]:
df01.plot(kind='scatter',x='dep_delay',y='arr_delay',color='red')
plt.show()

In [ ]:
df01.plot(kind='scatter',x='is_weekend',y='arr_delay',color='red')
plt.show()

# Exploratory Analisis

In [ ]:
#Plot delays on departures vs arrivals
f, (ax,ax1) = plt.subplots(1,2, figsize=(12,6))
dep = sns.countplot(df_delayed['dep_del15'], ax=ax)
dep.set_title('Depatures')
dep.set_xlabel('Delayed15')
dep.set_ylabel('Freq')

arr = sns.countplot(df_delayed['arr_del15'], ax=ax1)
arr.set_title('Arrivals')
arr.set_xlabel('Delayed15')
arr.set_ylabel('Freq')

In [ ]:
df_delayed.dtypes

In [ ]:
#Dep delay vs arr delay
sns.relplot(x="dep_delay", y="arr_delay", data=df_delayed);

In [ ]:
#Dep delay vs arr delay
sns.lmplot(x="dep_delay", y="arr_delay",hue="dep_time_blk",data=df_delayed);

In [ ]:
sns.distplot(df_delayed['arr_del15'])

In [ ]:
sns.regplot(data=df_delayed,
         y='dep_del15',
         x='distance',
         x_jitter=.1)

In [ ]:
#Plot delays on departures vs arrivals
f, (ax,ax1) = plt.subplots(1,2, figsize=(12,6))
dep = sns.countplot(df_delayed['origin'], ax=ax)
arr = sns.countplot(df_delayed['dest'], ax=ax1)


In [ ]:
#Plot delays on departures vs arrivals
fig, axs = plt.subplots(ncols=5, figsize=(30,5))
sns.violinplot(x="survived", y="age", hue="dep_del15", data=data, ax=axs[0])
sns.pointplot(x="sibsp", y="survived", hue="dep_del15", data=data, ax=axs[1])
sns.pointplot(x="parch", y="survived", hue="dep_del15", data=data, ax=axs[2])
sns.pointplot(x="pclass", y="survived", hue="dep_del15", data=data, ax=axs[3])
sns.violinplot(x="survived", y="fare", hue="dep_del15", data=data, ax=axs[4])


In [ ]:
sns.pointplot(x="origin", y="dep_del15", data=df_delayed)

In [ ]:
#Checking the distribution of delayed and not delayed
print(df_delayed['dep_del15'].value_counts())
print(df_delayed.shape)

In [ ]:
f,ax= plt.subplots(figsize=(15,15))
sns.heatmap(df_delayed.corr(),linewidths=.5,annot=True,fmt='.4f',ax=ax)
plt.show()

In [ ]:
df_carrier = pd.DataFrame(df_delayed['op_carrier'].value_counts().reset_index().values, columns=["op_carrier", "AggregateOP"])
df_carrier= df_carrier.sort_values('AggregateOP',ascending=False)
#df_carrier = df_carrier.sort_values('AggregateOP',ascending=True)
df_carrier

In [ ]:
print(df_delayed['dep_del15'].value_counts())
df_carrier_delayed = df_delayed[df_delayed['dep_del15'] ==True]
df_carrier_delayed = pd.DataFrame(df_carrier_delayed['op_carrier'].value_counts().reset_index().values, columns=["op_carrier", "AggregateOP"])
df_carrier_delayed

In [ ]:
# creating subplots 
ax = plt.subplots() 
sns.set_style('dark')
sns.set_palette('pastel')
# plotting columns 
ax = sns.barplot(x='op_carrier', y='AggregateOP',data=df_carrier ,color='g') 
ax = sns.barplot(x='op_carrier', y='AggregateOP',data=df_carrier_delayed, color='r') 
  
# renaming the axes 
ax.set(xlabel="x-axis", ylabel="y-axis") 
  
# visulaizing illustration 
plt.show() 

In [ ]:
sns.set_style('dark')
sns.set_palette('bright')
sns.barplot(x="op_carrier", y="AggregateOP", data=df_carieer)

In [ ]:
df_carieer = df_carieer.sort_values('AggregateOP',ascending=True)
df_carieer.shape

In [ ]:
df_carieer = pd.DataFrame(df_delayed['op_carrier'].value_counts().reset_index().values, columns=["op_carrier", "AggregateOP"])
df_carieer= df_carieer.sort_values('AggregateOP',ascending=False)

fig = px.bar(df_carieer, y='AggregateOP', x='op_carrier', text='AggregateOP', opacity = 0.8)
fig.update_traces(texttemplate='%{text:.1s}', textposition='outside')
fig.update_layout(uniformtext_minsize=4, uniformtext_mode='hide', title_x = 0.5,title_text='Carriers Count (OP Carrier)', yaxis=dict(title='Count'),
                  xaxis=dict(title='OP Carrier Code'))
fig.add_annotation( x='WN', y=100000, text="Highest OP CARRIER - WN",showarrow=True, font=dict( family="Courier New, monospace", size=10, color="#ffffff" ), align="right", arrowhead=2, arrowsize=1, arrowwidth=2, arrowcolor="#636363", ax=120, ay=0, bordercolor="#c7c7c7", borderwidth=2, borderpad=4, bgcolor="#ff7f0e", opacity=0.8)

fig.add_trace(go.Scatter(x=df_carieer['op_carrier'], y=df_carieer['AggregateOP'],
                    mode='lines+markers',opacity = 0.3,showlegend=False,
                   line = dict(
        smoothing = 1.2, color = 'blue',
        shape = "spline"
    )))

fig.show()

In [ ]:
# Review delays vs Departure time block
dep_time_blk = df_delayed[['dep_time_blk','dep_del15']].groupby('dep_time_blk').sum().sort_values(by='dep_del15',ascending=False)
dep_time_blk['percent'] = dep_time_blk['dep_del15']/(dep_time_blk['dep_del15'].sum())*100
print(dep_time_blk)
print('Total records: '+str(dep_time_blk['dep_del15'].sum()))

In [ ]:
# Review departure delays vs arrival time block
arr_time_blk = df_delayed[['arr_time_blk','arr_del15']].groupby('arr_time_blk').sum().sort_values(by='arr_del15',ascending=False)
arr_time_blk['percent'] = arr_time_blk['arr_del15']/(arr_time_blk['arr_del15'].sum())*100
print(arr_time_blk)
print('Total records: '+str(arr_time_blk['arr_del15'].sum()))

In [ ]:
print(df_delayed['day_of_week'].value_counts())
print(df_delayed.shape)

In [ ]:
#Delayed vs weekday
weekday = df_delayed[['day_of_week','dep_del15']].groupby('day_of_week').sum().sort_values(by='dep_del15',ascending=False)
weekday['percent'] = weekday['dep_del15']/(weekday['dep_del15'].sum())*100
print(weekday)
print('Total records: '+str(weekday['dep_del15'].sum()))

In [ ]:
#Delayed vs Monthday
monthday = df_delayed[['day_of_month','dep_del15']].groupby('day_of_month').sum().sort_values(by='dep_del15',ascending=False)
monthday['percent'] = monthday['dep_del15']/(monthday['dep_del15'].sum())*100
print(monthday)
print('Total records: '+str(monthday['dep_del15'].sum()))

In [ ]:
#Delayed vs origin airport
origin = df_delayed[['origin','origin_top10','dep_del15']].groupby(['origin','origin_top10']).sum().sort_values(by='dep_del15',ascending=False)
origin['percent'] = origin['dep_del15']/(origin['dep_del15'].sum())*100
#print(origin)
print(origin[0:9])
#print('Total records: '+str(origin['dep_del15'].sum()))

In [ ]:
dest = df_delayed[['dest','arr_del15','dest_top10']].groupby(['dest','dest_top10']).sum().sort_values(by='arr_del15',ascending=False)
dest['PERCENTUAL'] = dest['arr_del15']/(dest['arr_del15'].sum())*100
dest.head(10)

In [ ]:
print(df_delayed['origin_top10'].value_counts())
print(df_delayed.shape)
#print(origin[700:])

In [ ]:
df_delayed.groupby('origin').sum()

In [ ]:
df_delayed['day_of_week'].value_counts()

In [ ]:
df_delayed[['origin','origin_top10']].value_counts()

In [ ]:
    88724+     82732+    104663+    111874+    109367+     68850+     84640

In [ ]:
df_delayed.shape

# Data set split

In [ ]:
y=df_delayed['arr_del15']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)